In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 

import numpy as np

batch_size = 128
test_size = 256
channel_depth = 3
num_catecories = 10

## A parameter representing how much max pool is reducing the complexity
## Example: 2 implies the image is halved in height/width
max_pool_reduction_ratio = 2 ####IMPORTANT: this ratio should evenly divide the image size

def init_weights(shape):
    return tf.Variable(tf.random_normal(shape, stddev=0.01))

def model(X, w, w_fc, w_o, p_keep_conv, p_keep_hidden):
    l1a = tf.nn.relu(tf.nn.conv2d(X, w,                       # l1a shape=(?, 28, 28, 32)
                        strides=[1, 1, 1, 1], padding='SAME'))
    l1 = tf.nn.max_pool(l1a, ksize=[1, max_pool_reduction_ratio, max_pool_reduction_ratio, 1],              # l1 shape=(?, 14, 14, 32)
                        strides=[1, 2, 2, 1], padding='SAME')
    l1 = tf.nn.dropout(l1, p_keep_conv)


    l3 = tf.reshape(l1, [-1, w_fc.get_shape().as_list()[0]])    # reshape to (?, 14x14x32)

    l4 = tf.nn.relu(tf.matmul(l3, w_fc))
    l4 = tf.nn.dropout(l4, p_keep_hidden)

    pyx = tf.matmul(l4, w_o)
    return pyx


 # Load data
def load_data(nb_classes=10):
    from keras.datasets import cifar10
    from keras.utils import np_utils
    from keras import backend
    backend.set_image_data_format('channels_last')   
    # the data, shuffled and split between train and test sets
    (X_train, y_train), (X_test, y_test) = cifar10.load_data()

    def prepare_X(X):
        return X.astype('float32') / 255

    X_train, X_test = [prepare_X(X) for X in (X_train, X_test)]
    print(X_train.shape[0], 'train image samples of shape: ', X_train.shape[1:])
    print(X_test.shape[0], 'test samples')

    # convert class vectors to binary class matrices (basically one hot)
    Y_train = np_utils.to_categorical(y_train, nb_classes)
    Y_test = np_utils.to_categorical(y_test, nb_classes)

    w = X_train[0].shape[0]
    h = X_train[0].shape[1]

    # Fixes datasets whose shape don't imply depth (example, MNIST shape is 28x28 instead of 28,28,1)
    if (np.size(X_train[0].shape) == 2):
      X_train = X_train.reshape(-1, w, h, channel_depth)  # 28x28x1 input img (in the case of MNIST)
      X_test = X_test.reshape(-1, w, h,   channel_depth)  # 28x28x1 input img (in the case of MNIST)    
    
    return X_train, Y_train, X_test, Y_test, (w,h)

trX, trY, teX, teY, shape = load_data()

# Lets load our placeholders automatically based on the shape of the dataset
X = tf.placeholder("float", [None, shape[0], shape[1], channel_depth])
Y = tf.placeholder("float", [None, num_catecories])

### This is the most important section. Here we decide the actual dimensions of our neural network
##  for simplicity, we will do just a 3x3 convolve, but we need to make sure the depth is included.
filter_size = [8,8]
filter_depth = channel_depth 
conv_depth= 64               # this is the number of layers we want to feed into the next step and is essentially how many (possibly colour) filters we want to find

num_outputs = 800            # number of outputs before the category outputs to give us tunable weight to increase the accuracy of the output layer
pool_dimensions = [conv_depth* int(shape[0]/max_pool_reduction_ratio) * int(shape[1]/max_pool_reduction_ratio),num_outputs]


w = init_weights([filter_size[0], filter_size[1], filter_depth, conv_depth])       # example 3x3x3x32 => 32, 3 channel filters. The window size is the 3x3
w_fc = init_weights(pool_dimensions)                                               # example: a 28x28 image with a 2 reduction ratio, becomes a 14x14 image in t
w_o = init_weights([num_outputs, num_catecories])                                  # FC 625 inputs, 10 outputs (labels)
p_keep_conv = tf.placeholder("float")
p_keep_hidden = tf.placeholder("float")
py_x = model(X, w, w_fc, w_o, p_keep_conv, p_keep_hidden)


cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=py_x, labels=Y))
train_op = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(cost)
predict_op = tf.argmax(py_x, 1)

# Launch the graph in a session
with tf.Session() as sess:
    # you need to initialize all variables
    tf.global_variables_initializer().run()

    for i in range(100):
        training_batch = zip(range(0, len(trX), batch_size),
                             range(batch_size, len(trX)+1, batch_size))
        for start, end in training_batch:
            sess.run(train_op, feed_dict={X: trX[start:end], Y: trY[start:end], p_keep_conv: 0.8, p_keep_hidden: 0.5})

        test_indices = np.arange(len(teX)) # Get A Test Batch
        np.random.shuffle(test_indices)
        test_indices = test_indices[0:test_size]

        print(i, np.mean(np.argmax(teY[test_indices], axis=1) ==
                         sess.run(predict_op, feed_dict={X: teX[test_indices],
                                                         p_keep_conv: 1.0,
                                                         p_keep_hidden: 1.0})))


Instructions for updating:
non-resource variables are not supported in the long term
170500096/170498071 [==============================] - 11s 0us/step
50000 train image samples of shape:  (32, 32, 3)
10000 test samples
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
0 0.34375
1 0.44921875
2 0.6015625
3 0.62109375
4 0.64453125
5 0.61328125
6 0.64453125
7 0.63671875
8 0.66015625
9 0.69921875
10 0.703125
11 0.671875
12 0.65234375
13 0.671875
14 0.69921875
15 0.64453125
16 0.67578125
17 0.66796875
18 0.6640625
19 0.6875
20 0.68359375
21 0.64453125
22 0.7265625
23 0.703125
24 0.65625
25 0.640625
26 0.734375
27 0.65625
2

In [ ]:
sess.run(predict_op, feed_dict={X: teX[test_indices],
                                                         p_keep_conv: 1.0,
                                                         p_keep_hidden: 1.0})